In [1]:
import optuna
from optuna import Trial

from math import sqrt
from typing import Tuple, List

import numpy as np
import pandas as pd
from mordred import Calculator, descriptors
#import openbabel
from openbabel import pybel
from PyBioMed.PyMolecule.fingerprint import CalculatePubChemFingerprint,CalculateECFP2Fingerprint
from rdkit import Chem
from rdkit.Chem.rdchem import Atom

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score, confusion_matrix, matthews_corrcoef, roc_curve, auc 
from sklearn.metrics import precision_recall_curve

from sklearn.model_selection import StratifiedKFold, KFold


from torch_geometric.data import Data
from torch_geometric.loader import DataLoader as G_Loader 
from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.nn import global_mean_pool
from torch_geometric.nn import BatchNorm


# RDkit
from rdkit import Chem
from rdkit.Chem.rdmolops import GetAdjacencyMatrix
# Pytorch and Pytorch Geometric
import torch

import torch.nn as nn
from torch.nn import Linear
import torch.optim as optim
import torch.nn.functional as F # activation function
from torch.utils.data import Dataset
from torch.utils.data import DataLoader as V_Loader # dataset management
%run ./graph_feature.ipynb 
%run ./dataset_processing.ipynb 

%run ./my_performances.ipynb 


C:\Users\USER\anaconda3\envs\toxpred\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#load data descriptor 
# reload the data fingerprint 
#=======================================================
k=10
final_clean_desc_train=[]
final_clean_desc_val=[]
for i in range(k):
    final_clean_desc_train.append(np.load('final_clean_desc_train'+ str(i)+'.npy'))
    final_clean_desc_val.append(np.load('final_clean_desc_val' +str(i)+'.npy'))

final_clean_desc_test = np.load('final_clean_desc_test.npy')

In [3]:
type(final_clean_desc_test)

numpy.ndarray

In [5]:
train_idx = np.load('train_indices.npy')
val_idx = np.load('val_indices.npy')
test_idx = np.load('test_indices.npy')

In [6]:
# load the output label 
total_train_targets =[]
total_validation_targets =[]
total_test_targets=[]
for i in range(k):
    total_train_targets.append(np.load('total_train_targets'+ str(i)+'.npy'))
    total_validation_targets.append(np.load('total_validation_targets' +str(i)+'.npy'))

total_test_targets= np.load('total_test_targets.npy')

In [7]:
# create dataloader for training (vector data)
#======================================================================================
list_data_desc_train =[]
list_data_desc_val =[]
list_data_target_train =[]
list_data_target_val =[]

for data_train, data_val, tr_targets, val_targets in zip(final_clean_desc_train, final_clean_desc_val,total_train_targets, total_validation_targets):
    train_loader = V_Loader(dataset = data_train, batch_size = 126)
    val_loader = V_Loader(dataset = data_val, batch_size = 126)
    
    tr_target_loader = V_Loader(dataset = tr_targets, batch_size = 126)
    val_target_loader =  V_Loader(dataset = val_targets, batch_size = 126)
    
    list_data_desc_train.append(train_loader)
    list_data_desc_val.append(val_loader)
    
    list_data_target_train.append(tr_target_loader)
    list_data_target_val.append(val_target_loader)

In [8]:
#criterion = torch.nn.CrossEntropyLoss()
#define the loss function 
criterion = torch.nn.BCELoss()

In [9]:
def train_1(train_v_loaderB,tr_target_v_loaderB, combined_model, optimizer):
    combined_model.train()

# only used for counting the lost function for data train
#====================================================================    
    
    for data_X2, data_target in zip(train_v_loaderB, tr_target_v_loaderB):  # Iterate in batches over the training dataset.
        out = combined_model(torch.tensor(data_X2, dtype=torch.float32))  # Perform a single forward pass.
        #print(out)
        y_t = torch.tensor(data_target, dtype=torch.float32)
        loss = criterion(out[:,0], y_t)  # Compute the loss.
        #print(k)
        #print(loss)
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.
    return loss, combined_model, optimizer

# only used for counting the lost function for data validation
#====================================================================
def validation_1(val_v_loaderB,val_target_v_loaderB, combined_model):
    for data_X2, data_target in zip(val_v_loaderB, val_target_v_loaderB): # Iterate in batches over the training dataset.
        out = combined_model(torch.tensor(data_X2, dtype=torch.float32))  # Perform a single forward pass.
        
        y_t = torch.tensor(data_target, dtype=torch.float32)
        val_loss = criterion(out[:,0], y_t)  # Compute the loss.
        
    return val_loss

In [16]:
from sklearn.metrics import accuracy_score, confusion_matrix, matthews_corrcoef, roc_curve, auc 
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score

# performances visualization 
import matplotlib.pyplot as plt
#import seaborn as sns
import statistics
import math
from prettytable import PrettyTable
%run ./my_performances.ipynb 


def test_1(v_loaderB,v_target, combined_model):
    combined_model.eval()
    list_pred =[]
    list_targets =[]
    correct = 0
    for data_X2, data_target in zip (v_loaderB,v_target):  # Iterate in batches over the training/test dataset.
            out = combined_model(torch.tensor(data_X2, dtype=torch.float32))
            out_1 = out[:,0]
            
            list_pred.append(out_1.item())
            list_targets.append(data_target.item())
    return list_pred, list_targets

# used to count the train accuracy ,and validation accuracy when in the training mode 
def test(v_loaderB,target_v_loaderB, combined_model):
    combined_model.eval()

    correct = 0
    for data_X2, data_target in zip(v_loaderB,target_v_loaderB):  # Iterate in batches over the training/test dataset.
            out = combined_model(torch.tensor(data_X2, dtype=torch.float32))
            out_1 = out[:,0]
            for i,value in enumerate(out_1) :
                if value > 0.5 :
                    out_1[i] = 1
                else : out_1[i] = 0
            pred = out_1  # Use the class with highest probability.
            correct += int((pred == data_target).sum())  # Check against ground-truth labels.
    return correct / len(v_loaderB.dataset)  # Derive ratio of correct predictions.

In [11]:
def get_optimizer(gnn_model, learning_rate, optimizer_type, weight_decay=1e-4):
    if optimizer_type==1:
        optimizer = torch.optim.SGD(gnn_model.parameters(), lr=learning_rate, momentum=0.9)
    if optimizer_type==2:
        optimizer = torch.optim.Adam(gnn_model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    if optimizer_type ==3 :
        optimizer = torch.optim.Adamax(gnn_model.parameters(), lr=learning_rate, betas=(0.9, 0.999), eps=1e-08, weight_decay=1e-4)
        
    return optimizer

In [12]:
class modelB(torch.nn.Module):
    def __init__(self, input_features, output_features,dropout_rateB1,dropout_rateB2,dropout_rateB3,  
                 dense_layer1,dense_layer2, dense_layer3):
        super(modelB, self).__init__()
        self.lin1 = nn.Linear(input_features,dense_layer1)
      
        self.lin2 = nn.Linear(int(dense_layer1), dense_layer2)
        self.lin3 = nn.Linear(int(dense_layer2), dense_layer3)
        self.lin4 = nn.Linear(int(dense_layer3), output_features)
        
        self.bn1 = nn.BatchNorm1d(int(dense_layer1))
        self.bn2 = nn.BatchNorm1d(int(dense_layer2))
        self.bn3 = nn.BatchNorm1d(int(dense_layer3))
        self.dropoutB1 = dropout_rateB1
        self.dropoutB2 = dropout_rateB2
        self.dropoutB3 = dropout_rateB3
        
    def forward(self, x):
        x = self.lin1(x)
        
        x = self.bn1(x)
       
        x = F.dropout(x, p= self.dropoutB1, training=self.training)
        x = x.relu()
  #      
        x = self.lin2(x)
        x = self.bn2(x)   
        x = F.dropout(x, p= self.dropoutB2, training=self.training)
        x = x.relu()
  #      
        x = self.lin3(x)
        x = self.bn3(x)   
        x = F.dropout(x, p= self.dropoutB3, training=self.training)

        x = x.relu()
        x = self.lin4(x)
        return torch.sigmoid(x)    

In [13]:
k=10 # number of fold 
input_featuresB    = 1613 # length of feature data vector 
output_featuresB   = 1
optimizer_type = 2
dropout_rateB1 = 0.15552924759769313
dropout_rateB2 =0.207127885808487
dropout_rateB3 =0.2240506664932075

dense_layer1=60
dense_layer2=30
dense_layer3=10
learning_rate = 0.015061165279742287
weight_decay=0.00017722962529205202

list_data_vec_trainB = list_data_desc_train
list_data_vec_valB = list_data_desc_val

In [14]:
# APPLY THE HYPERPARAMETER 
# RUNNING EVERY FOLD 
#=============================================================

total_history_train_loss=[]
total_history_val_loss=[]
total_history_train_acc=[]
total_history_val_acc=[]
list_trained_model =[]

for i in range(k):
        
        combined_model   = modelB(input_featuresB, output_featuresB,
                                   dropout_rateB1,dropout_rateB1,dropout_rateB3, 
                                   dense_layer1,dense_layer2, dense_layer3)
        optimizer = get_optimizer(combined_model, learning_rate, optimizer_type,weight_decay)
   
        history_train_loss=[]
        history_val_loss=[]
        history_train_acc=[]
        history_val_acc=[]
        
        for epoch in range(1,40):
            
                
            train_loss,combined_model, optimizer = train_1(list_data_vec_trainB[i],list_data_target_train[i],
                                                           combined_model,optimizer)
            val_loss                             = validation_1(list_data_vec_valB[i],list_data_target_val[i],
                                                                combined_model) 
        
            train_acc                            = test(list_data_vec_trainB[i],list_data_target_train[i],combined_model)
            val_acc                              = test(list_data_vec_valB[i],list_data_target_val[i], combined_model)
        
           
            
            history_train_loss.append(train_loss.item())
            history_val_loss.append(val_loss.item())
            history_train_acc.append(train_acc)
            history_val_acc.append(val_acc)
        
            print(f'Fold: {i}')
            print(f'Epoch: {epoch:03d}, Train Loss:, {train_loss.item():.4f}')
            print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}')
            print(f'Epoch: {epoch:03d}, val Loss:, {val_loss.item():.4f}')
            print(f'Epoch: {epoch:03d}, val Acc: {val_acc:.4f}')
            print(" ")
        total_history_train_loss.append(history_train_loss)
        total_history_val_loss.append(history_val_loss)
        total_history_train_acc.append(history_train_acc)
        total_history_val_acc.append(history_val_acc)

        list_trained_model.append(combined_model)


C:\Users\USER\anaconda3\envs\toxpred\lib\site-packages\ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
C:\Users\USER\anaconda3\envs\toxpred\lib\site-packages\ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.
C:\Users\USER\anaconda3\envs\toxpred\lib\site-packages\ipykernel_launcher.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\USER\anaconda3\envs\toxpred\lib\site-packages\ipykernel_launcher.py:25: UserWarning: To copy construct f

Fold: 0
Epoch: 001, Train Loss:, 0.5935
Epoch: 001, Train Acc: 0.6246
Epoch: 001, val Loss:, 0.6063
Epoch: 001, val Acc: 0.5952
 
Fold: 0
Epoch: 002, Train Loss:, 0.5383
Epoch: 002, Train Acc: 0.6723
Epoch: 002, val Loss:, 0.6033
Epoch: 002, val Acc: 0.6825
 
Fold: 0
Epoch: 003, Train Loss:, 0.5145
Epoch: 003, Train Acc: 0.6758
Epoch: 003, val Loss:, 0.6004
Epoch: 003, val Acc: 0.6825
 
Fold: 0
Epoch: 004, Train Loss:, 0.5230
Epoch: 004, Train Acc: 0.6837
Epoch: 004, val Loss:, 0.5762
Epoch: 004, val Acc: 0.6984
 
Fold: 0
Epoch: 005, Train Loss:, 0.4746
Epoch: 005, Train Acc: 0.6829
Epoch: 005, val Loss:, 0.5455
Epoch: 005, val Acc: 0.7143
 
Fold: 0
Epoch: 006, Train Loss:, 0.4548
Epoch: 006, Train Acc: 0.7200
Epoch: 006, val Loss:, 0.5499
Epoch: 006, val Acc: 0.7063
 
Fold: 0
Epoch: 007, Train Loss:, 0.4490
Epoch: 007, Train Acc: 0.7067
Epoch: 007, val Loss:, 0.6223
Epoch: 007, val Acc: 0.7143
 
Fold: 0
Epoch: 008, Train Loss:, 0.4430
Epoch: 008, Train Acc: 0.6405
Epoch: 008, val Loss

Fold: 1
Epoch: 027, Train Loss:, 0.2439
Epoch: 027, Train Acc: 0.8489
Epoch: 027, val Loss:, 0.7889
Epoch: 027, val Acc: 0.7381
 
Fold: 1
Epoch: 028, Train Loss:, 0.1959
Epoch: 028, Train Acc: 0.8189
Epoch: 028, val Loss:, 0.8397
Epoch: 028, val Acc: 0.7460
 
Fold: 1
Epoch: 029, Train Loss:, 0.1789
Epoch: 029, Train Acc: 0.6546
Epoch: 029, val Loss:, 0.9626
Epoch: 029, val Acc: 0.5397
 
Fold: 1
Epoch: 030, Train Loss:, 0.1829
Epoch: 030, Train Acc: 0.8631
Epoch: 030, val Loss:, 0.9431
Epoch: 030, val Acc: 0.7302
 
Fold: 1
Epoch: 031, Train Loss:, 0.1947
Epoch: 031, Train Acc: 0.8525
Epoch: 031, val Loss:, 0.8860
Epoch: 031, val Acc: 0.7937
 
Fold: 1
Epoch: 032, Train Loss:, 0.1830
Epoch: 032, Train Acc: 0.8525
Epoch: 032, val Loss:, 0.7803
Epoch: 032, val Acc: 0.7540
 
Fold: 1
Epoch: 033, Train Loss:, 0.1853
Epoch: 033, Train Acc: 0.7986
Epoch: 033, val Loss:, 0.9581
Epoch: 033, val Acc: 0.7063
 
Fold: 1
Epoch: 034, Train Loss:, 0.2074
Epoch: 034, Train Acc: 0.8463
Epoch: 034, val Loss

Fold: 3
Epoch: 015, Train Loss:, 0.3431
Epoch: 015, Train Acc: 0.6431
Epoch: 015, val Loss:, 0.9754
Epoch: 015, val Acc: 0.5794
 
Fold: 3
Epoch: 016, Train Loss:, 0.3042
Epoch: 016, Train Acc: 0.7836
Epoch: 016, val Loss:, 1.0102
Epoch: 016, val Acc: 0.6270
 
Fold: 3
Epoch: 017, Train Loss:, 0.2335
Epoch: 017, Train Acc: 0.8039
Epoch: 017, val Loss:, 1.0045
Epoch: 017, val Acc: 0.6190
 
Fold: 3
Epoch: 018, Train Loss:, 0.3462
Epoch: 018, Train Acc: 0.8092
Epoch: 018, val Loss:, 0.9442
Epoch: 018, val Acc: 0.5476
 
Fold: 3
Epoch: 019, Train Loss:, 0.2716
Epoch: 019, Train Acc: 0.8313
Epoch: 019, val Loss:, 0.8824
Epoch: 019, val Acc: 0.6905
 
Fold: 3
Epoch: 020, Train Loss:, 0.2931
Epoch: 020, Train Acc: 0.8154
Epoch: 020, val Loss:, 0.9643
Epoch: 020, val Acc: 0.6825
 
Fold: 3
Epoch: 021, Train Loss:, 0.2792
Epoch: 021, Train Acc: 0.4594
Epoch: 021, val Loss:, 0.9300
Epoch: 021, val Acc: 0.4048
 
Fold: 3
Epoch: 022, Train Loss:, 0.3371
Epoch: 022, Train Acc: 0.5910
Epoch: 022, val Loss

Fold: 5
Epoch: 004, Train Loss:, 0.4567
Epoch: 004, Train Acc: 0.6820
Epoch: 004, val Loss:, 0.6449
Epoch: 004, val Acc: 0.6746
 
Fold: 5
Epoch: 005, Train Loss:, 0.4530
Epoch: 005, Train Acc: 0.7138
Epoch: 005, val Loss:, 0.6495
Epoch: 005, val Acc: 0.6667
 
Fold: 5
Epoch: 006, Train Loss:, 0.4394
Epoch: 006, Train Acc: 0.7341
Epoch: 006, val Loss:, 0.7228
Epoch: 006, val Acc: 0.6270
 
Fold: 5
Epoch: 007, Train Loss:, 0.4191
Epoch: 007, Train Acc: 0.7491
Epoch: 007, val Loss:, 0.7201
Epoch: 007, val Acc: 0.6667
 
Fold: 5
Epoch: 008, Train Loss:, 0.3952
Epoch: 008, Train Acc: 0.6820
Epoch: 008, val Loss:, 0.7251
Epoch: 008, val Acc: 0.6111
 
Fold: 5
Epoch: 009, Train Loss:, 0.4099
Epoch: 009, Train Acc: 0.7774
Epoch: 009, val Loss:, 0.7309
Epoch: 009, val Acc: 0.6667
 
Fold: 5
Epoch: 010, Train Loss:, 0.3946
Epoch: 010, Train Acc: 0.7394
Epoch: 010, val Loss:, 0.7216
Epoch: 010, val Acc: 0.6349
 
Fold: 5
Epoch: 011, Train Loss:, 0.3719
Epoch: 011, Train Acc: 0.7977
Epoch: 011, val Loss

Fold: 6
Epoch: 030, Train Loss:, 0.2057
Epoch: 030, Train Acc: 0.7208
Epoch: 030, val Loss:, 2.2848
Epoch: 030, val Acc: 0.5000
 
Fold: 6
Epoch: 031, Train Loss:, 0.3144
Epoch: 031, Train Acc: 0.7871
Epoch: 031, val Loss:, 1.9886
Epoch: 031, val Acc: 0.6587
 
Fold: 6
Epoch: 032, Train Loss:, 0.2642
Epoch: 032, Train Acc: 0.8534
Epoch: 032, val Loss:, 1.9787
Epoch: 032, val Acc: 0.5714
 
Fold: 6
Epoch: 033, Train Loss:, 0.2297
Epoch: 033, Train Acc: 0.8339
Epoch: 033, val Loss:, 1.5400
Epoch: 033, val Acc: 0.5397
 
Fold: 6
Epoch: 034, Train Loss:, 0.1912
Epoch: 034, Train Acc: 0.8878
Epoch: 034, val Loss:, 1.4293
Epoch: 034, val Acc: 0.5873
 
Fold: 6
Epoch: 035, Train Loss:, 0.1973
Epoch: 035, Train Acc: 0.8198
Epoch: 035, val Loss:, 1.3562
Epoch: 035, val Acc: 0.6429
 
Fold: 6
Epoch: 036, Train Loss:, 0.2134
Epoch: 036, Train Acc: 0.8242
Epoch: 036, val Loss:, 1.2819
Epoch: 036, val Acc: 0.5476
 
Fold: 6
Epoch: 037, Train Loss:, 0.1657
Epoch: 037, Train Acc: 0.8401
Epoch: 037, val Loss

Fold: 8
Epoch: 017, Train Loss:, 0.2932
Epoch: 017, Train Acc: 0.8023
Epoch: 017, val Loss:, 0.7974
Epoch: 017, val Acc: 0.6320
 
Fold: 8
Epoch: 018, Train Loss:, 0.3193
Epoch: 018, Train Acc: 0.7643
Epoch: 018, val Loss:, 1.0137
Epoch: 018, val Acc: 0.6400
 
Fold: 8
Epoch: 019, Train Loss:, 0.3367
Epoch: 019, Train Acc: 0.8058
Epoch: 019, val Loss:, 0.9924
Epoch: 019, val Acc: 0.5680
 
Fold: 8
Epoch: 020, Train Loss:, 0.3289
Epoch: 020, Train Acc: 0.5296
Epoch: 020, val Loss:, 1.0500
Epoch: 020, val Acc: 0.4960
 
Fold: 8
Epoch: 021, Train Loss:, 0.3430
Epoch: 021, Train Acc: 0.7176
Epoch: 021, val Loss:, 1.0440
Epoch: 021, val Acc: 0.6800
 
Fold: 8
Epoch: 022, Train Loss:, 0.2250
Epoch: 022, Train Acc: 0.8597
Epoch: 022, val Loss:, 1.0822
Epoch: 022, val Acc: 0.6640
 
Fold: 8
Epoch: 023, Train Loss:, 0.3124
Epoch: 023, Train Acc: 0.7705
Epoch: 023, val Loss:, 1.2180
Epoch: 023, val Acc: 0.5120
 
Fold: 8
Epoch: 024, Train Loss:, 0.2913
Epoch: 024, Train Acc: 0.8252
Epoch: 024, val Loss

In [17]:
# RUNNING THE 10 PRETRAINED MODEL USING INDEPENDENT DATA
#======================================================================
nCV= 10 # ten crossfold validation 
list_fold_pred =[]
list_fold_targets =[]


v_test_loaderB = V_Loader(dataset = final_clean_desc_test, batch_size = 1)
v_test_target = V_Loader(dataset = total_test_targets, batch_size = 1)

for combined_model in list_trained_model:  
    list_pred, list_targets = test_1(v_test_loaderB,v_test_target,combined_model)
    list_fold_pred.append(list_pred)
    list_fold_targets.append(list_targets)
    


C:\Users\USER\anaconda3\envs\toxpred\lib\site-packages\ipykernel_launcher.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [18]:
# GET THE PERFORMANCES FROM THE TEST
#========================================================================
total_performances = performances(list_fold_pred, list_fold_targets, nCV)
list_bal_acc = []
for sen, spec in zip (total_performances[1] , total_performances[2]):
    bal_acc = (sen + spec)/2
    list_bal_acc.append(bal_acc)
                
statistics.mean(list_bal_acc)

0.618155

In [19]:
import statistics
from prettytable import PrettyTable
perf = total_performances
model_title = 'Test Perf FCNN'
data_type ='Descriptor'
Create_Tables(perf, model_title, data_type)

+----------------+------------+-------+-------+-------+-------+-------+---------+---------+-----------+-------+
|   Model Name   | Data Type  | m_ACC |  m_SN |  m_SP | m_MCC | m_AUC | m_Kappa | m_AUROC | m_Bal_ACC |  m_F1 |
+----------------+------------+-------+-------+-------+-------+-------+---------+---------+-----------+-------+
| Test Perf FCNN | Descriptor | 0.625 | 0.637 | 0.599 | 0.241 | 0.685 |  0.219  |  0.685  |   0.618   | 0.685 |
+----------------+------------+-------+-------+-------+-------+-------+---------+---------+-----------+-------+
+----------------+------------+-------+-------+-------+-------+-------+---------+---------+-----------+-------+
|   Model Name   | Data Type  | e_ACC |  e_SN |   SP  | e_MCC | e_AUC | e_Kappa | e_AUROC | e_Bal_ACC |  e_F1 |
+----------------+------------+-------+-------+-------+-------+-------+---------+---------+-----------+-------+
| Test Perf FCNN | Descriptor | 0.082 | 0.199 | 0.188 | 0.047 | 0.022 |   0.06  |  0.022  |   0.024   | 